In [1]:
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

import re,os
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\linkr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_json('../data/newsgroups.json')
print(df.target_names.unique())
df.head(20)

['rec.autos' 'comp.sys.mac.hardware' 'rec.motorcycles' 'misc.forsale'
 'comp.os.ms-windows.misc' 'alt.atheism' 'comp.graphics'
 'rec.sport.baseball' 'rec.sport.hockey' 'sci.electronics' 'sci.space'
 'talk.politics.misc' 'sci.med' 'talk.politics.mideast'
 'soc.religion.christian' 'comp.windows.x' 'comp.sys.ibm.pc.hardware'
 'talk.politics.guns' 'talk.religion.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstei...,8,rec.motorcycles
100,From: tchen@magnus.acs.ohio-state.edu (Tsung-K...,6,misc.forsale
1000,From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)\n...,2,comp.os.ms-windows.misc
10000,From: a207706@moe.dseg.ti.com (Robert Loper)\n...,7,rec.autos
10001,From: kimman@magnus.acs.ohio-state.edu (Kim Ri...,6,misc.forsale
10002,From: kwilson@casbah.acns.nwu.edu (Kirtley Wil...,2,comp.os.ms-windows.misc
10003,Subject: Re: Don't more innocents die without ...,0,alt.atheism
10004,From: livesey@solntze.wpd.sgi.com (Jon Livesey...,0,alt.atheism


In [3]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]


In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_post', 'host', 'rac_wam', 'umd', 'organization', 'university', 'maryland_college', 'park', 'line', 'wonder', 'anyone', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'whatev', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


In [9]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [10]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.094*"space" + 0.044*"image" + 0.025*"display" + 0.021*"blue" + '
  '0.019*"earth" + 0.018*"print" + 0.017*"research" + 0.016*"project" + '
  '0.016*"satellite" + 0.015*"tank"'),
 (1,
  '0.050*"food" + 0.045*"bus" + 0.044*"specifically" + 0.036*"msg" + '
  '0.030*"eat" + 0.030*"motorcycle" + 0.025*"ride" + 0.018*"confuse" + '
  '0.016*"entry" + 0.016*"shift"'),
 (2,
  '0.029*"use" + 0.027*"system" + 0.021*"window" + 0.018*"card" + 0.016*"file" '
  '+ 0.016*"run" + 0.014*"program" + 0.013*"problem" + 0.012*"also" + '
  '0.011*"need"'),
 (3,
  '0.080*"game" + 0.076*"team" + 0.045*"play" + 0.045*"win" + 0.044*"player" + '
  '0.036*"hockey" + 0.030*"season" + 0.026*"contact" + 0.020*"goal" + '
  '0.020*"fan"'),
 (4,
  '0.112*"circuit" + 0.018*"stable" + 0.012*"cam" + 0.011*"chemistry" + '
  '0.010*"scout" + 0.007*"tran" + 0.006*"arabic" + 0.005*"randomly" + '
  '0.004*"axis" + 0.000*"tablet"'),
 (5,
  '0.129*"gun" + 0.052*"_" + 0.036*"weapon" + 0.025*"discipline" + '
  '0.023*"cri

In [11]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -14.792846211728248

Coherence Score:  0.5025181203779839


In [12]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\linkr\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      26.783260        1       1 -0.361838 -0.070396
13     22.667957        1       2 -0.353335 -0.124222
12     12.953420        1       3 -0.296767 -0.075629
2      10.931305        1       4 -0.267226 -0.027500
15      8.618549        1       5 -0.205847  0.115209
6       5.042087        1       6 -0.094983  0.441079
17      3.256525        1       7 -0.138357 -0.079402
3       1.988042        1       8 -0.025691  0.031646
0       1.898948        1       9  0.026194  0.030530
16      1.504172        1      10  0.041435  0.018807
5       1.296325        1      11  0.130654 -0.026183
8       0.645863        1      12  0.137391 -0.009258
18      0.562712        1      13  0.145351 -0.015357
1       0.504953        1      14  0.153874 -0.020526
11      0.369607        1      15  0.170989 -0.025606
9       0.297865        1      16  0.178092 -0.028677
19      0.295467        1      17  0.182552 -0.031821
10      0.248325        1      18  0.176186 -0.028067
4       0.074328        1      19  0.199575 -0.036906
14      0.060280        1      20  0.201750 -0.037720, topic_info=      Category          Freq                 Term         Total  loglift  \
term                                                                       
48396  Default  64902.000000                   ax  64902.000000  30.0000   
20777  Default  24205.000000                  not  24205.000000  29.0000   
65012  Default  18730.000000                 line  18730.000000  28.0000   
26018  Default  16710.000000         organization  16710.000000  27.0000   
2811   Default  15422.000000                   do  15422.000000  26.0000   
23934  Default   7809.000000           university   7809.000000  25.0000   
47206  Default  13744.000000                write  13744.000000  24.0000   
11254  Default  11807.000000                   be  11807.000000  23.0000   
61850  Default  17232.000000                would  17232.000000  22.0000   
22344  Default   4706.000000                  max   4706.000000  21.0000   
60178  Default   6772.000000            nntp_post   6772.000000  20.0000   
51531  Default   6320.000000                 host   6320.000000  19.0000   
42650  Default  10769.000000              article  10769.000000  18.0000   
70778  Default   2698.000000                  gun   2698.000000  17.0000   
35117  Default   5516.000000                 year   5516.000000  16.0000   
70576  Default   2883.000000                space   2883.000000  15.0000   
26020  Default  10381.000000                  say  10381.000000  14.0000   
29258  Default   5392.000000               system   5392.000000  13.0000   
18018  Default  10702.000000                 know  10702.000000  12.0000   
18769  Default   7222.000000                  use   7222.000000  11.0000   
9491   Default   7960.000000                 good   7960.000000  10.0000   
33016  Default   2570.000000                 game   2570.000000   9.0000   
39801  Default   2443.000000                 team   2443.000000   8.0000   
12811  Default   1482.000000                drive   1482.000000   7.0000   
52230  Default   8830.000000                   go   8830.000000   6.0000   
22717  Default   2031.000000                  key   2031.000000   5.0000   
56475  Default   3697.000000                thank   3697.000000   4.0000   
16684  Default   2371.000000            christian   2371.000000   3.0000   
28706  Default   3680.000000               window   3680.000000   2.0000   
955    Default   7793.000000                think   7793.000000   1.0000   
...        ...           ...                  ...           ...      ...   
25390  Topic20      9.239209        demonstration     10.356581   7.2998   
57218  Topic20      5.612239  bush_administration      6.729610   7.2324   
37355  Topic20      4.667923        qualification      5.785295   7.1993   
43265  Topic20      0.011483     

In [17]:
# import os
# os.environ['MALLET_HOME'] = 'C:\\Users\\linkr\\Documents\\TopicModelling\\mallet_dir'

# # mallet_path = Path("C:", "Users", "linkr", "Documents","TopicModelling", "mallet_dir", "bin", "mallet")
# mallet_path = 'C:\\Users\\linkr\\Documents\\TopicModelling\\mallet_dir\\bin\\mallet'
# ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [43]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,7.0,0.5861,"not, do, would, write, be, article, know, line...",From: (wheres my thing) Subject: WHAT car is t...
1,1,15.0,0.5868,"line, organization, university, host, nntp_pos...",From: (Guy Kuo) Subject: SI Clock Poll - Final...
2,2,7.0,0.5253,"not, do, would, write, be, article, know, line...",From: (Irwin Arnstein) Subject: Re: Recommenda...
3,3,2.0,0.3601,"use, system, window, card, file, run, program,...",From: (Tsung-Kun Chen) Subject: ** Software fo...
4,4,7.0,0.5109,"not, do, would, write, be, article, know, line...",From: (Don A.B. Lindbergh) Subject: Diamond SS...
5,5,7.0,0.5208,"not, do, would, write, be, article, know, line...",From: (Robert Loper) Subject: Re: SHO and SC N...
6,6,15.0,0.3115,"line, organization, university, host, nntp_pos...",From: (Kim Richard Man) Subject: SyQuest 44M c...
7,7,15.0,0.4628,"line, organization, university, host, nntp_pos...",From: (Kirtley Wilson) Subject: Mirosoft Offic...
8,8,13.0,0.2492,"people, make, say, may, state, would, many, fa...",Subject: Re: Dont more innocents die without t...
9,9,13.0,0.5072,"people, make, say, may, state, would, many, fa...",From: (Jon Livesey) Subject: Re: Genocide is C...


In [45]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0.0,0.2975,"space, image, display, blue, earth, print, res...",From: (D.V.Prakash) Subject: Pointer Feature S...
1,2.0,0.6649,"use, system, window, card, file, run, program,...",From: (GRUBB) Subject: Re: IDE vs SCSI Organiz...
2,3.0,0.4865,"game, team, play, win, player, hockey, season,...",Subject: NHL Summary parse results for games p...
3,5.0,0.7388,"gun, _, weapon, discipline, criminal, crime, t...",Subject: roman.bmp 01/14In response to the req...
4,6.0,0.9977,"ax, max, prove, count, somewhere, announce, pr...",Subject: roman.bmp 12/14 From: (Cliff) Reply-T...


In [49]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics.sort_values(['Num_Documents'], ascending=False)
# df_dominant_topics.Num_Documents.sum()

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
7,15.0,"line, organization, university, host, nntp_pos...",7242.0,0.6401
13,7.0,"not, do, would, write, be, article, know, line...",1896.0,0.1676
2,7.0,"not, do, would, write, be, article, know, line...",925.0,0.0818
15,7.0,"not, do, would, write, be, article, know, line...",742.0,0.0656
12,7.0,"not, do, would, write, be, article, know, line...",347.0,0.0307
3,2.0,"use, system, window, card, file, run, program,...",51.0,0.0045
17,13.0,"people, make, say, may, state, would, many, fa...",43.0,0.0038
16,7.0,"not, do, would, write, be, article, know, line...",20.0,0.0018
0,7.0,"not, do, would, write, be, article, know, line...",15.0,0.0013
6,15.0,"line, organization, university, host, nntp_pos...",11.0,0.0010
